In [359]:
import numpy as np
import spoonacular as sp
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import requests
import urllib3
from bs4 import BeautifulSoup
api = sp.API("6bfbc007a4msh40ca7f6c3a5248ep15f018jsn23a63e9d66ec")

# Functions to find ingredients and calculate carbon footprint

In [360]:
def find_recipe(keyword):
    url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/recipes/search"

    querystring = {"query":keyword}

    headers = {
        'x-rapidapi-key': "6bfbc007a4msh40ca7f6c3a5248ep15f018jsn23a63e9d66ec",
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    text =response.text

    start = text.find('d')
    end = text.find(',')
    food_id = response.text[start+3:end]
    return food_id

In [361]:
def extract_info(text,target):
    result = ""
    cur = ''
    start = text.find(target)+len(target)+2
    while cur !=',':
        cur = text[start]
        result += cur
        start+=1
    return result
    

In [362]:
def get_gredient(food_id):
    url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/recipes/"+str(food_id)+"/ingredientWidget.json"

    headers = {
        'x-rapidapi-key': "6bfbc007a4msh40ca7f6c3a5248ep15f018jsn23a63e9d66ec",
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
        }

    response = requests.request("GET", url, headers=headers)
    mytext = response.text
    text_list = mytext.split("}}}")
    
    ingred = []
    
    col=["name","value","unit"]
    for s in text_list:
        try:
            row = [extract_info(s,c) for c in col]
            ingred.append(row)

        except:

            continue
        
    ingred_list = []
    
    for ing in ingred:
        name = ing[0].split('"')[1]
        amount =ing[1].split(',')[0]
        unit = ing[2].split('"')[1]
        ingred_list.append([name,amount,unit])
        
        
        
    ingred_gram = {}
    l = len(ingred_list)
    
    for i in range(1,l-1):
        try:
            mul = dictionary[ingred_list[i][2].lower()]

            amount = mul* float(ingred_list[i][1])
            ingred_gram[ingred_list[i][0]]=amount
        except:
            print("Unit dictionary doesn't have the unit:",ingred_list[i][2])

        
        
    return ingred_gram

In [363]:
def cal_footprint(ingred_list,threshold):
    score = 0
    for key,value in ingred_gram.items():

        match = process.extract(key,food_list,limit=1)    
        if match[0][1] >= threshold:
            score += value*dictionary3[match[0][0]]
        else:
            print('Could not find footprint of',key, 'from database. The cloest ingredient is:',match[0][0],
                  "with footprint ",dictionary3[match[0][0]],"gCO2 eq per gram")

    print("\nThe carbon footprint of your foot product is:",score,"gCO2eq")
    return score

In [364]:
def carbon_footprint_calculator(keyword,searchtype="rough"):
    food_id = find_recipe(keyword)
    gre_list = get_gredient(food_id)
    if searchtype=="rough":
        threshold = 60
    elif searchtype == "accurate":
        threshold = 80
    elif searchtype == "vague":
        threshold = 30
    score = cal_footprint(gre_list,threshold)
    return score

In [365]:
carbon_footprint_calculator('fried rice')

Unit dictionary doesn't have the unit: 
Unit dictionary doesn't have the unit: medium
Unit dictionary doesn't have the unit: large
Unit dictionary doesn't have the unit: 
Could not find footprint of red onion from database. The cloest ingredient is: Rapeseed Oil with footprint  3.8 gCO2 eq per gram
Could not find footprint of cilantro from database. The cloest ingredient is: Dark Chocolate with footprint  46.7 gCO2 eq per gram
Could not find footprint of plain yogurt from database. The cloest ingredient is: Cane Sugar with footprint  3.2 gCO2 eq per gram
Could not find footprint of kosher salt from database. The cloest ingredient is: Other Fruit with footprint  1.1 gCO2 eq per gram
Could not find footprint of fresh dill from database. The cloest ingredient is: Rapeseed Oil with footprint  3.8 gCO2 eq per gram
Could not find footprint of paprika from database. The cloest ingredient is: Rice with footprint  4.5 gCO2 eq per gram
Could not find footprint of cayenne from database. The cloes

876.8299999999999

# Database for unit convertion and GHG of ingredients


In [366]:
##construct GHG dictionary base
## roughly match items but dont explicitly say that.
data = pd.read_csv("Poore&Nemececk_GHG.csv")

In [367]:
data.head()

,Unnamed: 0,"Resampled, Randomized Data",Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54
0,NaN,Land Use (m2/FU),NaN,NaN,NaN,NaN,NaN,"GHG Emissions (kg CO2eq/FU, IPCC 2013 incl. CC...",NaN,NaN,...,"GHG (kg CO2eq/FU, IPCC 2013 incl. CC feedbacks)",NaN,"Acid. (kg SO2eq/FU, CML2 Baseline)",NaN,"Eutr. (kg PO43-eq/FU, CML2 Baseline)",NaN,Fresh W. (L/FU),NaN,Strs. W. WU (L/FU),NaN
1,Product,5th pctl,10th pctl,Mean,Median,90th pctl,95th pctl,5th pctl,10th pctl,Mean,...,Min,Max,Min,Max,Min,Max,Min,Max,Min,Max
2,Wheat & Rye (Bread),1.0,1.1,3.9,2.7,7.9,10.0,0.7,0.8,1.6,...,0.4,56.0,3.4,33.5,0.2,77.4,1,8675,1,333260
3,Maize (Meal),1.0,1.1,2.9,1.8,5.7,9.0,0.7,0.7,1.7,...,0.5,72.3,2.9,25.5,0.8,20.8,0,7191,0,461216
4,Barley (Beer),0.2,0.3,1.1,0.9,2.4,2.9,0.6,0.7,1.2,...,0.4,2.6,4.8,38.4,1.0,12.4,6,569,4,41484


In [368]:
glist = [{i:j} for i,j in zip(data.iloc[2:-2,0],data.iloc[2:-2,9])]

In [369]:
GHG_dictionary ={}
for ele in glist:
    for k,v in ele.items():
        GHG_dictionary[k]=float(v)

In [378]:
url = "https://www.visualcapitalist.com/visualising-the-greenhouse-gas-impact-of-each-food/"
headers={'user-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.192 Safari/537.36'}
result = requests.get(url,headers = headers)


In [379]:
result.content.decode()

'<!DOCTYPE html>\n<html lang="en-US">\n<head>\n<meta charset="UTF-8" >\n<meta name="viewport" id="viewport" content="width=device-width" />\n<link rel="shortcut icon" href="https://www.visualcapitalist.com/wp-content/uploads/2018/12/favicon1.ico" /><link rel="pingback" href="https://www.visualcapitalist.com/xmlrpc.php" />\n<meta property="og:type" content="article" />\n\t\t\t\t\t<meta name="twitter:image" content="https://www.visualcapitalist.com/wp-content/uploads/2020/02/carbon-footprint-food-prev-1000x600.png" />\n\t<meta property="og:url" content="https://www.visualcapitalist.com/visualising-the-greenhouse-gas-impact-of-each-food/" />\n<meta property="og:title" content="The Carbon Footprint of the Food Supply Chain" />\n<meta property="og:description" content="According to the largest ever meta-analysis of food systems, the carbon footprint of different types of food in your diet can vary widely." />\n<meta name="twitter:card" content="summary">\n<meta name="twitter:url" content="h

In [380]:
result.content.decode().find("tb")

105928

In [381]:
result.content.decode()[105928+1:].find('tbody')

3017

In [382]:
mytarget = result.content.decode()[105928+1:3017+105928+1]

In [383]:
dictionary2 = {}
target_list = mytarget.split('row')              
for s in target_list[2:]:
     
    key= s.split('column')[1].split('>')[1][:-4]
    value =s.split('column')[2].split('>')[1][:-4]
    dictionary2[key] = value


    

In [387]:
temp ={}
for key,value in dictionary2.items():
    temp[key] = float(value[:-6])

In [389]:
for key,value in temp.items():
    if key not in GHG_dictionary:
        GHG_dictionary[key]=value

In [390]:
GHG_dictionary

{'Wheat & Rye (Bread)': 1.6,
 'Maize (Meal)': 1.7,
 'Barley (Beer)': 1.2,
 'Oatmeal': 2.5,
 'Rice': 4.5,
 'Potatoes': 0.5,
 'Cassava': 1.3,
 'Cane Sugar': 3.2,
 'Beet Sugar': 1.8,
 'Other Pulses': 1.8,
 'Peas': 1.0,
 'Nuts': 0.4,
 'Groundnuts': 3.2,
 'Soymilk': 1.0,
 'Tofu': 3.2,
 'Soybean Oil': 6.3,
 'Palm Oil': 7.3,
 'Sunflower Oil': 3.6,
 'Rapeseed Oil': 3.8,
 'Olive Oil': 5.4,
 'Tomatoes': 2.1,
 'Onions & Leeks': 0.5,
 'Root Vegetables': 0.4,
 'Brassicas': 0.5,
 'Other Vegetables': 0.5,
 'Citrus Fruit': 0.4,
 'Bananas': 0.9,
 'Apples': 0.4,
 'Berries & Grapes': 1.5,
 'Wine': 1.8,
 'Other Fruit': 1.1,
 'Coffee': 28.5,
 'Dark Chocolate': 46.7,
 'Bovine Meat (beef herd)': 99.5,
 'Bovine Meat (dairy herd)': 33.3,
 'Lamb & Mutton': 39.7,
 'Pig Meat': 12.3,
 'Poultry Meat': 9.9,
 'Milk': 3.2,
 'Cheese': 23.9,
 'Eggs': 4.7,
 'Fish (farmed)': 13.6,
 'Crustaceans (farmed)': 26.9,
 'Beef (beef herd)': 60.0,
 'Lamb &amp; Mutton': 24.0,
 'Beef (dairy herd)': 21.0,
 'Chocolate': 19.0,
 'Prawns 

In [391]:
##convert unit 
unit_dictionary = {
    "g":1,
    "kg":1000,
    "lb":453.59,
    "lbs":453.59,
    "pound":454,
    "pounds":454,
    "oz":28.35,
    "ozs":28.35,
    "cup":125,
    "cups":125,
    "ml":1,
    "l":1000,
    "liter":1000,
    "liters":1000,
    "tsp":5,
    "tsps":5,
    "tbsp":14.79,
    "tbsps":14.79,
    "qt":946.35,
    "bunch":200,
    "rip":-1,
    "scoop":15,
    "scoops":15,
    "leaf":4,
    "leaves":4,
    "drop":0.05,
    "drops":0.05,
    "sheet":0.648,
    "sheets":0.648,
    "slice":25,
    "slices":25,
    "inch":16.39,
    "inches":16.39,
    "stalk":114,
    "stalks":114,
    "stick":113.4,
    "sticks":113.4,
    "strip":12,
    "strips":12,
    "sprig":3,
    "sprigs":3,
    "dash":0.711,
    "dashes":0.711,
    "pinch":0.356,
    "pinches":0.356,
    "can":400,
    "serving":75,
}